In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
INDOX_API_KEY = os.getenv("INDOX_API_KEY")

In [2]:
from indoxJudge.metrics import Gruen

In [3]:
from indoxJudge.models import IndoxApi
model = IndoxApi(api_key=INDOX_API_KEY)

In [4]:
response = ["The Mediterranean diet is known for its health benefits, including reducing the risk of heart disease, stroke, and diabetes. It encourages the consumption of fruits, vegetables, whole grains, nuts, and olive oil, while limiting red meat. Additionally, this diet has been associated with better cognitive function and a reduced risk of Alzheimer's disease, promoting longevity and overall well-being."]

In [5]:
gruen = Gruen(candidates=response)

In [6]:
from indoxJudge.piplines import CustomEvaluator
eval = CustomEvaluator(model=model,metrics=[gruen])

INFO: Evaluator initialized with model and metrics.
INFO: Model set for all metrics.


In [7]:
result = eval.judge()

INFO: Evaluating metric: Gruen


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 1/1 [00:00<00:00,  1.34it/s]
E:\Codes\IndoxJudge\indoxJudge\.venv\Lib\site-packages\transformers\data\processors\glue.py:66: FutureWarning: This function will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script f

INFO: Completed evaluation for metric: Gruen


In [8]:
result

{'gruen': {'score': [0.72]}}